In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-02-25 06:00:00')


In [3]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/436375
Connected. Call `.close()` to terminate connection gracefully.
FROM-TO DATA:
 2024-01-28 06:00:00+00:00 2024-02-25 05:00:00+00:00 



Fetching data from 2024-01-28 06:00:00+00:00 to 2024-02-25 05:00:00+00:00
Finished: Reading data from Hopsworks, using ArrowFlight (10.99s) 
178080 178080
                pickup_hour  rides  pickup_location_id      pickup_ts
0 2024-01-29 16:00:00+00:00      2                   4  1706544000000
1 2024-01-30 22:00:00+00:00      0                  67  1706652000000
2 2024-01-29 20:00:00+00:00      0                 220  1706558400000
3 2024-01-30 22:00:00+00:00      5                 265  1706652000000
5 2024-01-28 06:00:00+00:00      0                 207  1706421600000
2024-01-28 06:00:00+00:00 2024-02-25 05:00:00+00:00


In [4]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/436375
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] bagging_fraction is set=0.5404721974859934, subsample=1.0 will be ignored. Current value: bagging_fraction=0.5404721974859934


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,18.0,2024-02-25 06:00:00
1,2,15.0,2024-02-25 06:00:00
2,3,17.0,2024-02-25 06:00:00
3,4,23.0,2024-02-25 06:00:00
4,5,15.0,2024-02-25 06:00:00
...,...,...,...
260,261,16.0,2024-02-25 06:00:00
261,262,11.0,2024-02-25 06:00:00
262,263,20.0,2024-02-25 06:00:00
263,264,13.0,2024-02-25 06:00:00


### Save these predictions in the feature store, so they can be later consumed by our Streamlit app

In [6]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/436375
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 0.00% |          | Rows 0/265 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/436375/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f8c837a3be0>, None)